In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from IPython.display import Image
%matplotlib inline
import warnings
warnings.simplefilter('ignore')

import selenium
from selenium import webdriver
import pandas as pd
import csv
import time
import urllib
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,ElementNotInteractableException,ElementNotVisibleException,TimeoutException,ElementClickInterceptedException

In [9]:
def review_links(driver,review_link_lis):
    for n in range(0,200):
        review_link_lis.append(driver.current_url)
#         time.sleep(0.6)
        try:
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//li[@class='a-last']"))).click()
        except NoSuchElementException as e:
            break
        except TimeoutException as e:
            break
        except StaleElementReferenceException as e:
            break
    return review_link_lis

def links_one_star(driver,review_link_lis):
#     time.sleep(0.3)
    driver.find_element_by_xpath("//div[@class='a-column a-span3 star-rating-select']/span").click()
    time.sleep(0.3)
    driver.find_element_by_xpath("//li[@aria-labelledby='star-count-dropdown_5']").click()
    lis=review_links(driver,review_link_lis)
    driver,lis=links_two_star(driver,lis)
    return driver,lis

def links_two_star(driver,review_link_lis):
#     time.sleep(0.3)
    driver.find_element_by_xpath("//div[@class='a-column a-span3 star-rating-select']/span").click()
    time.sleep(0.3)
    driver.find_element_by_xpath("//li[@aria-labelledby='star-count-dropdown_4']").click()
    lis=review_links(driver,review_link_lis)
    driver,lis=links_three_star(driver,lis)
    return driver,lis

def links_three_star(driver,review_link_lis):
#     time.sleep(0.3)
    driver.find_element_by_xpath("//div[@class='a-column a-span3 star-rating-select']/span").click()
    time.sleep(0.3)
    driver.find_element_by_xpath("//li[@aria-labelledby='star-count-dropdown_3']").click()
    lis=review_links(driver,review_link_lis)
    driver,lis=links_four_star(driver,lis)
    return driver,lis

def links_four_star(driver,review_link_lis):
#     time.sleep(0.3)
    driver.find_element_by_xpath("//div[@class='a-column a-span3 star-rating-select']/span").click()
    time.sleep(0.3)
    driver.find_element_by_xpath("//li[@aria-labelledby='star-count-dropdown_2']").click()
    lis=review_links(driver,review_link_lis)
    driver,lis=links_five_star(driver,lis)
    return driver,lis

def links_five_star(driver,review_link_lis):
    time.sleep(0.3)
    driver.find_element_by_xpath("//div[@class='a-column a-span3 star-rating-select']/span").click()
    time.sleep(0.3)
    driver.find_element_by_xpath("//li[@aria-labelledby='star-count-dropdown_1']").click()
    lis=review_links(driver,review_link_lis)
    return driver,lis


def scrap_data_from_amazon(path,web_link,prod_name):
    driver = webdriver.Chrome(path)
    driver.get(web_link)
    search_box=driver.find_element_by_id('twotabsearchtextbox')
    search_box.send_keys(prod_name)
    driver.find_element_by_id('nav-search-submit-button').click()
    lis_brand_links=[]
    values={
            'Rating_heading':[],
            'Reviews':[],
            'Rating':[]
           }
    for page in range(1,3):
        for i in driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']"):
            lis_brand_links.append(i.get_attribute("href"))
        driver.find_element_by_xpath("//li[@class='a-last']").click()
        try:
            WebDriverWait(driver,5).until(EC.presence_of_element_located((By.CLASS_NAME,'a-last')))
        except NoSuchElementException as e:
            print(e)   
    for link in lis_brand_links:
        driver.get(link)
        try:
            try:
                driver.find_element_by_xpath("//a[@class='a-link-emphasis a-text-bold']").click()
            except NoSuchElementException as e:
                continue
            time.sleep(4)
            review_link_lis=[]
            driver,review_link_lis=links_one_star(driver,review_link_lis)
            for link in review_link_lis:
                driver.get(link)
                for t in driver.find_elements_by_xpath("//div[@class='a-section celwidget']"):
                    try:
                        t.find_element_by_xpath("div[5]/div/div/span/a").click()
                        time.sleep(0.2)
                    except NoSuchElementException as e:
                        pass
                    try:
                        time.sleep(0.2)
                        values['Rating'].append(t.find_element_by_xpath("div[2]/a").get_attribute('title'))
                        values['Reviews'].append(t.text.split("\n")[4])
                        values['Rating_heading'].append(t.text.split("\n")[1])
                    except StaleElementReferenceException as e:
                        continue
                    except NoSuchElementException as e:
                        continue
                        
        except TimeoutException as  e:
            time.sleep(2)
    return values



In [281]:
# prod_name=input("Enter the Product that you want to search : ")
data=scrap_data_from_amazon("/home/pavankumar/Downloads/chromedriver_linux64/chromedriver","https://www.amazon.in/",'laptops')
laptops=pd.DataFrame(data)
laptops.shape

In [252]:
laptops.to_csv("amazon_laptops.csv")

In [10]:
data=scrap_data_from_amazon("/home/pavankumar/Downloads/chromedriver_linux64/chromedriver","https://www.amazon.in/",'phones')
phones=pd.DataFrame(data)
phones.shape

(2348, 3)

In [11]:
phones.to_csv("amazon_phones.csv")

In [12]:
data=scrap_data_from_amazon("/home/pavankumar/Downloads/chromedriver_linux64/chromedriver","https://www.amazon.in/",'headphones')
headphones=pd.DataFrame(data)
headphones.shape

In [13]:
headphones.to_csv("amazon_headphones.csv")

In [14]:
data=scrap_data_from_amazon("/home/pavankumar/Downloads/chromedriver_linux64/chromedriver","https://www.amazon.in/",'Smart Watches')
smart_watches=pd.DataFrame(data)
smart_watches.shape

In [15]:
smart_watches.to_csv("amazon_smart_watches.csv")

In [18]:
data=scrap_data_from_amazon("/home/pavankumar/Downloads/chromedriver_linux64/chromedriver","https://www.amazon.in/",'Professional Cameras')
professional_cameras=pd.DataFrame(data)
professional_cameras.shape

(1115, 3)

In [19]:
professional_cameras.to_csv("amazon_professional_cameras.csv")

In [ ]:
data=scrap_data_from_amazon("/home/pavankumar/Downloads/chromedriver_linux64/chromedriver","https://www.amazon.in/",'Printers')
printers=pd.DataFrame(data)
printers.shape

In [ ]:
printers.to_csv("amazon_printers.csv")

In [16]:
data=scrap_data_from_amazon("/home/pavankumar/Downloads/chromedriver_linux64/chromedriver","https://www.amazon.in/",'monitors')
monitors=pd.DataFrame(data)
monitors.shape

(2182, 3)

In [20]:
monitors.to_csv("amazon_monitors.csv")

In [ ]:
data=scrap_data_from_amazon("/home/pavankumar/Downloads/chromedriver_linux64/chromedriver","https://www.amazon.in/",'Home Theater')
home_theater=pd.DataFrame(data)
home_theater.shape

In [ ]:
home_theater.to_csv("amazon_home_theater.csv")

In [ ]:
data=scrap_data_from_amazon("/home/pavankumar/Downloads/chromedriver_linux64/chromedriver","https://www.amazon.in/",'Routers')
router=pd.DataFrame(data)
router.shape

In [ ]:
router.to_csv("amazon_router.csv")

In [ ]:
amazone_data=pd.concat([laptops,phones,headphones,smart_watches,professional_cameras,printers,monitors.home_theater,router],axis=0)

In [ ]:
amazone_data.to_csv("amazon_all_data.csv")

### From flipkart

In [7]:
def append_reviews(driver,values,typ):
    for n in range(0,25):
        try:
            for a in driver.find_elements_by_xpath("//span[@class='_1BWGvX']"):  ## Read more
                a.click()
                time.sleep(0.5)
        except:
            pass
        for i in  driver.find_elements_by_xpath("//div[@class='col _2wzgFH K0kLPL']"):
            try:
                values["Rating"].append(i.text.split('\n')[:-4][0])
                values["Rating_heading"].append(i.text.split('\n')[:-4][1])
                values["Reviews"].append(' '.join(i.text.split('\n')[:-4][2:]))
            except:
                continue

        try:
            driver.find_element_by_xpath("//a[@class='_1LKTO3'][2]").click()
            time.sleep(2)
        except NoSuchElementException as e:
            try:
                if driver.find_element_by_xpath("//a[@class='_1LKTO3']").text=='NEXT':
                    driver.find_element_by_xpath("//a[@class='_1LKTO3']").click()
                    time.sleep(2)
                else:
                    break
            except:
                continue
        except ElementClickInterceptedException as e:
            try:
                if driver.find_element_by_xpath("//a[@class='_1LKTO3']").text=='NEXT':
                    driver.find_element_by_xpath("//a[@class='_1LKTO3']").click()
                    time.sleep(2)
                else:
                    break
            except:
                continue
    return driver,values


def extract_review(driver,values,lis_brand_links):
    for link in lis_brand_links:
        driver.get(link)
        try:
            try:
                driver.find_element_by_xpath("//div[@class='_3UAT2v _16PBlm']").click()  ## All reviews
            except NoSuchElementException as e:
                values['Reviews'].append("-")
                continue
        except TimeoutException as  e:
            time.sleep(0.2)
        time.sleep(1)
        typ='positive'
        driver.find_element_by_xpath("//select[@class='_1EDlbo tVKh2S']/option[3]").click()  ## Positive reviews
        time.sleep(1)
        driver,values=append_reviews(driver,values,typ)
        typ=='negative'
        driver.find_element_by_xpath("//select[@class='_1EDlbo tVKh2S']/option[4]").click()  ## Negative reviews
        time.sleep(1)
        driver,values=append_reviews(driver,values,typ)    

    return driver, values


def scrap_data_from_flipkart(path,web_link,prod_name):
    driver = webdriver.Chrome(path)
    driver.get(web_link)
    driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']").click()
    driver.find_element_by_xpath("//input[@class='_3704LK']").send_keys(prod_name)
    driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
    lis_brand_links=[]
    values={
            'Rating_heading':[],
            'Reviews':[],
            'Rating':[]
           }
    for page in range(1,3):  ## Links for number of pages
        for i in driver.find_elements_by_xpath("//a[@class='s1Q9rs']"):
            lis_brand_links.append(i.get_attribute("href"))
        for j in driver.find_elements_by_xpath("//a[@class='_1fQZEK']"):
            lis_brand_links.append(j.get_attribute("href"))
        try:
            if driver.find_element_by_xpath("//div[@class='_2MImiq']/nav/a[11]").text=="NEXT":
                driver.find_element_by_xpath("//div[@class='_2MImiq']/nav/a[11]").click()
                time.sleep(3)
            else:
                driver.find_element_by_xpath("//div[@class='_2MImiq']/nav/a[12]").click()
                time.sleep(3)
        except:
            time.sleep(3)
    driver,values=extract_review(driver,values,lis_brand_links)
    return values

global

In [18]:
prod_name=input("Enter the Product that you want to search : ")
data=scrap_data_from_flipkart("/home/pavankumar/Downloads/chromedriver_linux64/chromedriver","https://www.flipkart.com/",prod_name)
pd.DataFrame(data)